In [3]:
import pandas as pd
from scipy.stats import chi2_contingency, ttest_ind
import matplotlib.pyplot as plt
import seaborn as sns
 
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
import numpy as np

import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
 

airbnb_data = pd.read_csv("cleaned_airbnb_Chicago.csv")

predicting revenue using xgboost

In [5]:
#Determine the effect on revenue by using interaction term of seasons and controllable factors as indepndent variables
#(seasons here represent periods for example: period 5 is season 3, period 6 is season 4 and so on)

# Function to calculate the season based on your criteria
def calculate_season(period):
    if period == 5:
        return 3
    elif period == 6:
        return 4
    else:
        # Assuming the periods are in order and cycle every four periods
        return (period - 7) % 4 + 1

# Apply the function to calculate the season for each row
airbnb_data['season'] = airbnb_data['superhost_period_all'].apply(calculate_season)

# Handle NaN values and convert boolean columns to integers
airbnb_data['Pets Allowed'] = airbnb_data['Pets Allowed'].fillna(0).astype(int)
airbnb_data['Instantbook Enabled'] = airbnb_data['Instantbook Enabled'].fillna(0).astype(int)

# Replace infinite values with NaN and fill NaN values for numerical columns
numerical_cols = airbnb_data.select_dtypes(include=[np.number]).columns.tolist()
airbnb_data[numerical_cols] = airbnb_data[numerical_cols].apply(lambda x: x.replace([np.inf, -np.inf], np.nan).fillna(x.mean()))

# Ensure all columns are numeric for regression analysis
for col in airbnb_data.columns:
    if airbnb_data[col].dtype == 'object':
        airbnb_data[col] = pd.to_numeric(airbnb_data[col], errors='coerce')  # Convert non-numeric columns to numeric, coercing errors to NaN

# Create interaction terms between season and other variables
interaction_variables = ['Nightly Rate', 'Cleaning Fee (USD)', 'Max Guests', 'Minimum Stay',
                         'Pets Allowed', 'Instantbook Enabled', 'available_days', 'Number of Photos', 'hostResponseAverage_pastYear']
for var in interaction_variables:
    airbnb_data[f'season_{var}_interaction'] = airbnb_data['season'] * airbnb_data[var]

# Fill any new NaNs introduced by interaction terms
airbnb_data = airbnb_data.apply(lambda x: x.replace([np.inf, -np.inf], np.nan).fillna(x.mean()), axis=0)

# Filter columns that start with 'season_'
interaction_columns = [col for col in airbnb_data.columns if col.startswith('season_')]

# Calculate Z-scores for numerical columns
z_scores = np.abs(stats.zscore(airbnb_data[numerical_cols]))

# Define a Z-score threshold (e.g., 3)
z_score_threshold = 3

# Filter outliers based on Z-scores
outliers = (z_scores > z_score_threshold).any(axis=1)
df_no_outliers = airbnb_data[~outliers]

# Select additional independent variables you want to include
additional_independent_variables = ['booked_days', 'Bedrooms', 'Number of Reviews', 'Rating Overall','occupancy_rate',
                                    'Nightly Rate', 'Cleaning Fee (USD)', 'Max Guests', 'Minimum Stay',
                                    'Pets Allowed', 'Instantbook Enabled', 'available_days', 'Number of Photos',
                                    'hostResponseAverage_pastYear']

# Combine interaction columns and additional independent variables
selected_features = interaction_columns + additional_independent_variables

# Select features and target variable
X = df_no_outliers[selected_features]
y = df_no_outliers['revenue']

# Split the data into training and validation sets (80% training, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the regression model on the training data
X_train = sm.add_constant(X_train)  # Adding a constant column for the intercept
model = sm.OLS(y_train, X_train).fit()

# Print the summary of the regression
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                revenue   R-squared:                       0.707
Model:                            OLS   Adj. R-squared:                  0.707
Method:                 Least Squares   F-statistic:                     3467.
Date:                Fri, 08 Dec 2023   Prob (F-statistic):               0.00
Time:                        17:35:06   Log-Likelihood:            -2.8758e+05
No. Observations:               33101   AIC:                         5.752e+05
Df Residuals:                   33077   BIC:                         5.754e+05
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                                                      coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------